In [3]:
import numpy as np
import scipy as sp
import pandas as pd

from tqdm import tqdm

import matplotlib.pyplot as plt
import matplotlib.dates as mdates


In [2]:
file_path = r'\\RISKLAB_NAS\database1\선물 체결틱데이터(2010.Q1~2023.Q4)\DFKNXTRDSHRTH_2022_Q1.dat'

col_names = [
    'TRADE_DATE', 'REGUL_OFFHR_TP_CD', 'ISIN_CODE', 'JONG_INDEX', 
    'TRD_PRC', 'TRDVOL', 'TRD_TP_CD', 'TRD_DD', 'TRD_TM', 
    'NBMM_TRD_PRC', 'FUTRMM_TRD_PRC', 'OPEN_PRICE', 'HIGH_PRICE', 
    'LOW_PRICE', 'LST_PRC', 'ACC_TRDVOL', 'ACC_AMT', 
    'LST_ASKBID_TP_CD', 'LP_HD_QTY', 'DATA_TYPE', 'MSG_SEQ', 'BRD_ID', 
    'SESSION_ID', 'DYNMC_UPLMTPRC', 'DYNMC_LWLMTPRC'
]

df_raw = pd.read_csv(
    file_path,
    sep='|',
    header=None,
    names=col_names,
    encoding='euc-kr',
    on_bad_lines='skip'
)


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_23832\3049676060.py:12: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv(


In [3]:
is_kospi_futures = df_raw['ISIN_CODE'].str.startswith('KR4101', na=False)
df_KOSPI200Futures_2022Q1 = df_raw[is_kospi_futures].copy()

rename_map = {
    'TRD_DD': 'trddate',
    'TRD_TM': 'trdtime',
    'TRD_PRC': 'price',
    'TRDVOL': 'volume',
    'LST_ASKBID_TP_CD': 'bidask'
}

df_KOSPI200Futures_2022Q1 = df_KOSPI200Futures_2022Q1[
    ['TRD_DD', 'TRD_TM', 'TRD_PRC', 'TRDVOL', 'LST_ASKBID_TP_CD']
    ].rename(columns=rename_map)

datetime_str = (df_KOSPI200Futures_2022Q1['trddate'].astype(str) + 
                df_KOSPI200Futures_2022Q1['trdtime'].astype(str).str.zfill(9))

df_KOSPI200Futures_2022Q1['trddatetime'] = pd.to_datetime(
    datetime_str, format='%Y%m%d%H%M%S%f'
    )

df_KOSPI200Futures_2022Q1 = df_KOSPI200Futures_2022Q1[['trddatetime', 'price', 'volume', 'bidask']]

df_KOSPI200Futures_2022Q1


,trddatetime,price,volume,bidask
0,2022-01-03 07:30:10.506,394.25,1,
1,2022-01-03 07:30:10.508,394.25,15,
2,2022-01-03 07:30:10.596,394.25,2,
3,2022-01-03 07:30:10.600,394.25,2,
4,2022-01-03 07:30:10.602,394.25,1,
...,...,...,...,...
4284060,2022-03-31 15:34:40.799,365.75,1,1
4284061,2022-03-31 15:34:40.883,365.75,4,2
4284062,2022-03-31 15:34:40.983,365.75,2,2
4284063,2022-03-31 10:38:20.534,368.00,350,


In [7]:
df_KOSPI200Futures_2022Q1 = df_KOSPI200Futures_2022Q1.set_index('trddatetime')

df_2022Q1_10sec = df_KOSPI200Futures_2022Q1.resample('5s').agg(
    price=('price', 'last'),
    volume=('volume', 'sum')
)

df_2022Q1_10sec = df_2022Q1_10sec.dropna(subset=['price'])

df_2022Q1_10sec


,price,volume
trddatetime,,
2022-01-03 07:30:10,394.25,121
2022-01-03 07:31:00,394.25,171
2022-01-03 07:31:05,394.25,1
2022-01-03 07:32:00,394.25,6
2022-01-03 07:35:00,394.25,410
...,...,...
2022-03-31 15:34:55,365.80,80
2022-03-31 15:36:05,365.96,120
2022-03-31 15:38:00,365.96,150


In [8]:
start_date = '2022-01-03'
end_date = '2022-01-05'
data_sample = df_KOSPI200Futures_2022Q1[start_date:end_date]

fig, axes = plt.subplots(
    2, 1,
    sharex=True,
    # figsize=(15, 8),
    gridspec_kw={'height_ratios': [3, 1]}
)
fig.suptitle('KOSPI 200 Futures')

axes[0].plot(
    data_sample.index, 
    data_sample['price'], 
    color='dodgerblue', 
    linewidth=1.2, 
    label='Price'
    )
axes[0].set_ylabel('Price')
axes[0].grid(True, linestyle='--', alpha=0.6)
axes[0].legend()

axes[1].bar(
    data_sample.index, 
    data_sample['volume'], 
    color='gray', 
    width=0.0001, 
    label='Volume'
    )
axes[1].set_ylim(0, data_sample['volume'].quantile(0.99))
axes[1].set_ylabel('Volume')
axes[1].set_xlabel('Time')
axes[1].grid(True, linestyle='--', alpha=0.6)
axes[1].legend()

date_format = mdates.DateFormatter('%Y-%m-%d %H:%M:%S')
axes[1].xaxis.set_major_formatter(date_format)
fig.autofmt_xdate() # Auto-rotates date labels for better fit.

plt.tight_layout(rect=[0, 0.03, 1, 0.97]) # Adjust for suptitle


KeyError: 'Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is not allowed.'

In [ ]:
df_KOSPI200Futures_2022Q1['2022-01-03':'2022-01-04']

,price,volume,bidask
trddatetime,,,
2022-01-03 07:30:10.506,394.25,1,
2022-01-03 07:30:10.508,394.25,15,
2022-01-03 07:30:10.596,394.25,2,
2022-01-03 07:30:10.600,394.25,2,
2022-01-03 07:30:10.602,394.25,1,
...,...,...,...
2022-03-31 15:34:40.799,365.75,1,1
2022-03-31 15:34:40.883,365.75,4,2
2022-03-31 15:34:40.983,365.75,2,2
